# I. Tweet Data Cleaning



In [1]:
## Tweet Table

In [2]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json
import time

In [3]:
#function to cheack whether 
def check_var(data, x):
    try:
        data[x]
        return True
    except:
        return False

In [4]:
%%time

#making mongoDB database and collection
client = MongoClient('localhost', 27017)
db = client['mydatabase']

want_drop = input()
if(want_drop=='yes'): ## if type yes, then it will drop tweet table from mongodb to prepare new importing.
    db.tweet.drop()
    
tweet = db['tweet']

yes
CPU times: user 6.89 ms, sys: 4.52 ms, total: 11.4 ms
Wall time: 2.48 s


In [5]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [6]:
#yelin directory '/Users/yelin/Downloads/corona-out-3/corona-out-3'
#Mo 'C:/Users/maxmo/OneDrive/Desktop/Rutgers/Spring2020/Data Management/corona-out-2'
# Hui Wang  file = '/Users/huiwang/Downloads/corona-out-2'
file = '/Users/huiwang/Downloads/corona-out-3'

In [7]:
%%time

#id, user_id, created_at, text, retweet_count, favorite_count, reply_count, hashtags
import json
with open(file, 'r', encoding = 'utf8') as f1 :
    for line in f1 :
        try :
            data = json.loads(line)
        except :
            continue
        
        hashtag = [] #to collect list of hashtags
        
        if (data['lang'] == 'en'):
            
            #case 1: check whether it is a retweet data or not
            if (check_var(data, 'retweeted_status')) :

                Created_at = pd.to_datetime(data['retweeted_status']['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['retweeted_status']['user']['id']
                ID = data['retweeted_status']['id']
                retweet_count = data['retweeted_status']['retweet_count']
                favorite_count = data['retweeted_status']['favorite_count']
                reply_count = data['retweeted_status']['reply_count']

                #check whether there is extension version of that tweet or not        
                if (check_var(data['retweeted_status'], 'extended_tweet')) :
                    Text = data['retweeted_status']['extended_tweet']['full_text']
                    for t in data['retweeted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else :
                    Text = data['retweeted_status']['text']
                    for t in data['retweeted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

                #check whether it is retweet of quoted tweet or not
                if (check_var(data['retweeted_status'], 'quoted_status')) :

                    #check whether there is extension version of the original tweet or not 
                    if (check_var(data['retweeted_status']['quoted_status'], 'extended_tweet')) :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['extended_tweet']['full_text']
                        for t in data['retweeted_status']['quoted_status']['extended_tweet']['entities']['hashtags'] :
                            hashtag.append(t['text'])                
                    else :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['text']
                        for t in data['retweeted_status']['quoted_status']['entities']['hashtags'] :
                            hashtag.append(t['text'])               

            #case 2: check whether it is a quote tweet data or not
            elif (check_var(data, 'quoted_status')) :
                Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                #check whether there is extension version of that quote tweet or not 
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text'] + ' || ' 
                else:
                    Text = data['text'] + ' || '

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

                #check whether there is extension version of the originl tweet or not 
                if (check_var(data['quoted_status'], 'extended_tweet')) :
                    Text = Text + data['quoted_status']['extended_tweet']['full_text']
                    for t in data['quoted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else:
                    Text = Text + data['quoted_status']['text']
                    for t in data['quoted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

            #if it does not belong to case 1 and case 2, it is an ordinary tweet data
            else :
                Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                #check whether there is extension version of that tweet or not  
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text']
                else :
                    Text = data['text']

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

            tw = {'id' : ID,
                  'user_id' : UserID,
                  'created_at' : Created_at,
                  'text' : Text,
                  'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count,
                  'reply_count' : reply_count,
                  'hashtags': hashtag
                 }

            #if the data is already in our database, we update it 
            if (tweet.count_documents({'id' : tw['id']}) > 0) :
                db.tweet.update_one({'id' : tw['id']},
                                    {'$set' : {'retweet_count' : tw['retweet_count'],
                                               'favorite_count' : tw['favorite_count'],
                                               'reply_count' : tw['reply_count']}})
            #if it is not, we insert it    
            else :
                tweet.insert_one(tw)
            

tweet.create_index([ ('id', 1) ])

CPU times: user 1min 52s, sys: 6.07 s, total: 1min 58s
Wall time: 18min 25s


'id_1'

In [10]:
tweet_table = pd.DataFrame(tweet.find())
tweet_table.shape

(19465, 9)





# II. User Data Cleaning





In [11]:
## User Table

In [12]:
import pandas as pd
import psycopg2
import json

In [13]:
##Creating the database
conn = psycopg2.connect("dbname=postgres port=5432 user=postgres password=password")
conn.set_session(autocommit = True)
cur = conn.cursor()

want_drop = input()
if(want_drop=='yes'): ## if type yes, then it will drop database from postgreSQL to prepare new importing.
    cur.execute("DROP DATABASE IF EXISTS twitter_users" )
    
cur.execute("CREATE DATABASE twitter_users" )
conn.commit()
cur.close()
conn.close()

yes


In [14]:
#Creating the table
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS users_data( user_id varchar(50) PRIMARY KEY, name varchar(255), user_name varchar(255), verified_status boolean,followers_count integer,friends_count integer,statuses_count integer,user_location varchar(500),favourites_count integer);" )
conn.commit()
cur.close()
conn.close()

In [15]:
def check_var(data,x):
    try:
        data[x]
        return True
    except:
        return False

In [16]:
%%time

#This function takes the the variable inputs and stores it into a PostgreSQL database
def store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count):
    conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
    cur = conn.cursor()
    insert_query = "INSERT INTO users_data (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET verified_status=EXCLUDED.verified_status, followers_count=EXCLUDED.followers_count, friends_count=EXCLUDED.friends_count, statuses_count=EXCLUDED.statuses_count, user_location=EXCLUDED.user_location, favourites_count=EXCLUDED.favourites_count"
    cur.execute(insert_query, (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location,favourites_count))
    conn.commit()
    cur.close()
    conn.close()
    
    
# Decoding the JSON file
with open(file, 'r', encoding = 'utf8') as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
        except:
            continue
        
        if (data['lang'] == 'en'): # only 
            
            if (check_var(data, 'retweeted_status')):

                user_id = data['retweeted_status']['user']['id_str']
                name = data['retweeted_status']['user']['name']
                user_name = data['retweeted_status']['user']['screen_name']
                verified_status = data['retweeted_status']['user']['verified']
                followers_count = data['retweeted_status']['user']['followers_count']
                friends_count = data['retweeted_status']['user']['friends_count']
                statuses_count = data['retweeted_status']['user']['statuses_count']
                user_location = data['retweeted_status']['user']['location']
                favourites_count= data['retweeted_status']['user']['favourites_count']


            else :
                user_id = data['user']['id_str']
                name = data['user']['name']
                user_name = data['user']['screen_name']
                verified_status = data['user']['verified']
                followers_count = data['user']['followers_count']
                friends_count = data['user']['friends_count']
                statuses_count = data['user']['statuses_count']
                user_location = data['user']['location']
                favourites_count= data['user']['favourites_count']


            #insert the data into the PostgreSQL database
            store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count)

CPU times: user 52.8 s, sys: 13.8 s, total: 1min 6s
Wall time: 12min 19s


In [20]:
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()


# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [22]:
user_table = create_pandas_table("SELECT * FROM users_data")
user_table.shape

(15673, 9)





# III. Searching Application





In [23]:
## Install PySpark
#!curl -O https://d3kbcqa49mib13.cloudfront.net/spark-2.2.0-bin-hadoop2.7.tgz
#!tar -xvf spark-2.2.0-bin-hadoop2.7.tgz

In [24]:
#Configure your python environment by uncommenting the line below.
#!pip install findspark

In [25]:
## Launch PySpark
import os
import findspark
os.environ["PYSPARK_PYTHON"] = "python3"
findspark.init("spark-2.2.0-bin-hadoop2.7",)

Import components of SparkSQL

In [26]:
from pyspark.sql import SparkSession, Column, Row, functions as F

Initialize the SparkSQL session which contains a basic Spark Context. This may take a few moments to launch the cluster of (typically 4 to 8 python jobs in the background). Note that you are running Spark locally (.master).

In [27]:
spark = (
    SparkSession.builder
        .master("local[*]")
        .appName("SparkLecture694Example")
        .getOrCreate()
)
sc = spark.sparkContext

In [28]:
# BUG tweet_table_spark = spark.createDataFrame(tweet_table) 

In [29]:
# BUG user_table_spark = spark.createDataFrame(user_table)

Implement 1

In [30]:
# Within the Tweet table, a user can get such as the most popular original tweet ranked by the retweet_count, favorite_count, and reply_count; 
# or the most popular hashtag ranked by aggregating the lists of hashtags among all tweets we collect


In [31]:
# using MongoDB
# most popular original tweet ranked by the retweet_count
pop_retweet = pd.DataFrame(tweet.find().sort("retweet_count",-1))
pop_retweet.head(5)

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea4ac7b221c62f93908afc4,1238264431320215553,1100261477989126145,"March 13, 2020, 12:43:40 AM",*corona virus enters my body*\n\nThe 4 Flintst...,237307,811062,1811,[]
1,5ea4ac32221c62f93908a913,1240334979701395458,1112592502727548928,"March 18, 2020, 05:51:18 PM",When this Corona shit passes we have to promis...,181584,764405,1595,[]
2,5ea4abce221c62f939089d91,1237436114887041024,1131227186,"March 10, 2020, 05:52:14 PM",THIS MAN IS A GENIUS he figured out the Corona...,179037,515867,1578,[]
3,5ea4acd0221c62f93908b562,1239689136358985729,728371324184301568,"March 16, 2020, 11:04:57 PM",“corona time “😭😭😭😭 https://t.co/iXBMHVcFoY,132003,422685,1516,[]
4,5ea4afc3221c62f93908d441,1242137430322446344,716432228,"March 23, 2020, 05:13:35 PM",This is Dr. Usama Riaz. He spent past weeks sc...,119601,331618,4876,[]


In [32]:
# using MongoDB 
# most popular original tweet ranked by the retweet_count, reply_count, and favorite_count 
pop_hottest = pd.DataFrame(tweet.find().sort([("retweet_count",-1),("reply_count",-1),("favorite_count",-1)]))
pop_hottest.head(5)

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea4ac7b221c62f93908afc4,1238264431320215553,1100261477989126145,"March 13, 2020, 12:43:40 AM",*corona virus enters my body*\n\nThe 4 Flintst...,237307,811062,1811,[]
1,5ea4ac32221c62f93908a913,1240334979701395458,1112592502727548928,"March 18, 2020, 05:51:18 PM",When this Corona shit passes we have to promis...,181584,764405,1595,[]
2,5ea4abce221c62f939089d91,1237436114887041024,1131227186,"March 10, 2020, 05:52:14 PM",THIS MAN IS A GENIUS he figured out the Corona...,179037,515867,1578,[]
3,5ea4acd0221c62f93908b562,1239689136358985729,728371324184301568,"March 16, 2020, 11:04:57 PM",“corona time “😭😭😭😭 https://t.co/iXBMHVcFoY,132003,422685,1516,[]
4,5ea4afc3221c62f93908d441,1242137430322446344,716432228,"March 23, 2020, 05:13:35 PM",This is Dr. Usama Riaz. He spent past weeks sc...,119601,331618,4876,[]


In [35]:
# using MongoDB 
# the most single popular hashtag ranked by aggregating the lists of hashtags
pop_hashtags = db.tweet.group(
   {
     key: { user_id: 1, hashtags: 1 },
     reduce: function( curr, result ){},
     initial: { }
   }
)

pop_hashtags.head(5)

SyntaxError: invalid syntax (<ipython-input-35-08c800b60da3>, line 6)

Implement 2

In [ ]:
# With the user table, a user can get such as the most active verified user ranked by statuses_count; 
# most popular user ranked by followers_count, friends_count, and favourites_count;
# or the most active location ranked by tweets count.

In [38]:
# using SQL
# the most active verified user ranked by statuses_count
active_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status='True' ORDER BY statuses_count DESC")
active_user.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,47596019,Liputan6.com,liputan6dotcom,True,3687560,693,1381273,Jakarta Indonesia,7062
1,16973333,The Independent,Independent,True,3202840,1146,976895,"London, England",3
2,378809160,𝔾𝕀𝔻𝕀𝕋ℝ𝔸𝔽𝔽𝕀ℂ,Gidi_Traffic,True,1676895,5826,922265,Everywhere,14478
3,2475273985,Paytm Care,Paytmcare,True,248675,15,861933,India,402
4,15448383,Inquirer,inquirerdotnet,True,3076056,1852,858348,Makati City,34260


In [42]:
# using SQL
# the most popular user ranked by followers_count, friends_count, and favourites_count
pop_user = create_pandas_table("SELECT * FROM (SELECT user_id, name, user_location, sum(followers_count+friends_count+favourites_count) as likes FROM users_data ORDER BY likes DESC) WHERE verified_status='True' ")
pop_user.head(5)

DatabaseError: Execution failed on sql 'SELECT * FROM (SELECT user_id, name, user_location, sum(followers_count+friends_count+favourites_count) as likes FROM users_data ORDER BY likes DESC) WHERE verified_status='True' ': subquery in FROM must have an alias
LINE 1: SELECT * FROM (SELECT user_id, name, user_location, sum(foll...
                      ^
HINT:  For example, FROM (SELECT ...) [AS] foo.


In [ ]:
# using SQL
# the most active location ranked by tweets count
pop_location = create_pandas_table("SELECT * FROM users_data WHERE verified_status='True' ORDER BY statuses_count DESC")
pop_location.head(5)

In [84]:
spark.sql("""
    SELECT user_id, statuses_count
    FROM user_table
    SORT BY statuses_count
""").toPandas()

AnalysisException: 'Table or view not found: user_table; line 3 pos 9'

In [37]:
# join the tweet table and user table

Implement 3

In [ ]:
# When a user searches for a keyword or hashtag in the Tweet table, the results will show all the original tweets 
# that contain it. We can rank it by number of retweets, number of favorites, number of replies, or sum of them to 
# find the most popular tweet. 

Implement 4

In [ ]:
# We can also link the results with the user table to find out the corresponding users 
# who tweet them and rank the results by the number of followers, number of friends, number of statuses, 
# number of favorites or combination of them.

Implement 5

In [ ]:
# We also save the location information of users in the user table, even though it is self-reported and users 
# can choose whether they want to register or not. We still plan to set the related search in our application. 

# A user can search for the most preferred location ranked by amount of twitter users. 
# When a user searches a hashtag, the results can show the most active location related to the hashtag.